In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import time
import json
import os

def load_class_labels():
    """Load class labels from file or use default labels"""
    try:
        with open('class_labels.json', 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        # Default pest labels if file not found
        return [
            "aphids", "armyworm", "beetle", "bollworm", "grasshopper",
            "mites", "mosquito", "sawfly", "stem_borer"
        ]

def predict_from_frame(frame, model, class_labels):
    """
    Predict pest class from a video frame using MobileNetV2
    
    Args:
        frame: Video frame from camera
        model: Loaded MobileNetV2 model
        class_labels: List of class labels
        
    Returns:
        Predicted class, confidence score, and processed frame
    """
    # Resize frame to match MobileNetV2 input size (224x224)
    img_height, img_width = 224, 224
    processed_frame = cv2.resize(frame, (img_width, img_height))
    
    # Convert from BGR to RGB (OpenCV uses BGR by default)
    processed_frame_rgb = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
    
    # Preprocess for MobileNetV2 input
    img_array = image.img_to_array(processed_frame_rgb)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Get predicted class and confidence
    predicted_class_idx = np.argmax(predictions[0])
    predicted_class = class_labels[predicted_class_idx]
    confidence = predictions[0][predicted_class_idx]
    
    return predicted_class, confidence, frame

def main():
    print("Loading MobileNetV2 pest detection model...")
    
    # Load MobileNetV2 model
    try:
        model = load_model('mobilenetv2_model.h5')
        print("Loaded MobileNetV2 model successfully")
    except:
        print("Error: MobileNetV2 model not found. Please ensure 'mobilenetv2_model.h5' exists.")
        return
    
    # Load class labels
    class_labels = load_class_labels()
    print(f"Loaded {len(class_labels)} class labels: {class_labels}")
    
    # Initialize camera
    print("Starting camera...")
    cap = cv2.VideoCapture(0)  # Use 0 for default camera
    
    if not cap.isOpened():
        print("Error: Could not open camera. Please check your camera connection.")
        return
    
    # Set camera resolution
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    print("Camera initialized. Press 'q' to quit, 'c' to capture and classify an image.")
    
    last_prediction_time = 0
    prediction_interval = 1.0  # Seconds between predictions to avoid overloading
    current_prediction = "No prediction yet"
    current_confidence = 0.0
    
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        if not ret:
            print("Error: Failed to grab frame")
            break
        
        # Make a copy of the frame for display
        display_frame = frame.copy()
        
        # Get current time
        current_time = time.time()
        
        # Process every prediction_interval seconds
        if current_time - last_prediction_time > prediction_interval:
            predicted_class, confidence, _ = predict_from_frame(frame, model, class_labels)
            current_prediction = predicted_class
            current_confidence = confidence
            last_prediction_time = current_time
        
        # Display the prediction on the frame
        text = f"{current_prediction}: {current_confidence:.2f}"
        cv2.putText(display_frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                    0.8, (0, 255, 0), 2)
        
        # Draw a bounding box to indicate where to position the pest
        h, w = display_frame.shape[:2]
        cv2.rectangle(display_frame, (w//4, h//4), (3*w//4, 3*h//4), (0, 255, 0), 2)
        
        # Display instructions
        cv2.putText(display_frame, "Press 'c' to capture", (10, h-40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(display_frame, "Press 'q' to quit", (10, h-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        # Display the resulting frame
        cv2.imshow('MobileNetV2 Pest Detection', display_frame)
        
        # Key handling
        key = cv2.waitKey(1) & 0xFF
        
        # Press 'q' to quit
        if key == ord('q'):
            break
        # Press 'c' to capture and get a detailed analysis
        elif key == ord('c'):
            # Show "Processing..." on frame
            processing_frame = display_frame.copy()
            cv2.putText(processing_frame, "Processing...", (w//2-100, h//2), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
            cv2.imshow('MobileNetV2 Pest Detection', processing_frame)
            cv2.waitKey(1)
            
            # Do a more careful prediction
            predicted_class, confidence, _ = predict_from_frame(frame, model, class_labels)
            
            # Display result
            print(f"\nDetailed Analysis:")
            print(f"Detected Pest: {predicted_class}")
            print(f"Confidence: {confidence:.4f}")
            
            # Save the captured image with prediction
            timestamp = time.strftime("%Y%m%d-%H%M%S")
            filename = f"pest_capture_{timestamp}_{predicted_class}.jpg"
            
            # Create 'captures' directory if it doesn't exist
            if not os.path.exists('captures'):
                os.makedirs('captures')
                
            cv2.imwrite(f"captures/{filename}", frame)
            print(f"Image saved as: captures/{filename}")
            
            # Get all class probabilities for detailed view
            img_height, img_width = 224, 224
            processed_image = cv2.resize(frame, (img_width, img_height))
            processed_image = cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB)
            img_array = image.img_to_array(processed_image)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            
            predictions = model.predict(img_array, verbose=0)
            
            print("\nAll class probabilities:")
            for i, (label, prob) in enumerate(zip(class_labels, predictions[0])):
                print(f"{label}: {prob:.4f}")
            print("\n")
    
    # Release camera and close windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Loading MobileNetV2 pest detection model...


Loaded MobileNetV2 model successfully
Loaded 9 class labels: ['aphids', 'armyworm', 'beetle', 'bollworm', 'grasshopper', 'mites', 'mosquito', 'sawfly', 'stem_borer']
Starting camera...
Camera initialized. Press 'q' to quit, 'c' to capture and classify an image.
